In [1]:
import sys
import sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Ще прочета тренировъчните данни и ще ги разгледам.

In [2]:
original = pd.read_csv('data/titanic/train.csv',index_col=['PassengerId'])
original.isnull().sum().sort_values()
data = original.copy()

print(data.shape)
data.head(10)

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S


In [3]:
data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
data.isnull().sum().sort_values()

Survived      0
Pclass        0
Name          0
Sex           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Embarked      2
Age         177
Cabin       687
dtype: int64

В Embaarked има 2 null стойности, които мисля, че няма особено значение с какво ще попълня. Има доста хора с неизвестна възраст и трябва да намеря подходящи стойности за тях. Повечето хора са с неизвестни кабини затова за сега ще игнорирам тази колона.

In [5]:
data.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

Ще запълня липсващите стойности в Embarked с произволно пристанище от възможните. Липсващите възрасти ще запълня със средната възраст за тази извадка.

In [6]:
data['Age'] = data.Age.fillna(data.Age.mean())
data['Embarked'] = data.Embarked.fillna('S')

data.isnull().sum().sort_values()

Survived      0
Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Embarked      0
Cabin       687
dtype: int64

Ще пробвам да тренирам логистична регресия само с числовите данни.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

X = data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin'], 1)
y = data['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

model = LogisticRegression(random_state=0).fit(X_train, y_train)
print("LogisticRegression: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))


LogisticRegression: 
train score: 0.708083832335
test score:  0.695067264574


Очаквано резултата не е много добър. Ще пробвам да добавя и пола към фичърите. Първо ще трябва да отделя двата пола в отделни колони.

In [8]:
data.Sex.unique()

array(['male', 'female'], dtype=object)

In [9]:
data['Male'] = (data.Sex == 'male').astype(float)
data['Female'] = (data.Sex == 'female').astype(float)
data.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Male,Female
PassengerId,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,1.0,0.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,0.0,1.0
3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,0.0,1.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,0.0,1.0
5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,1.0,0.0
6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q,1.0,0.0
7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,S,1.0,0.0
8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S,1.0,0.0
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,NaN,S,0.0,1.0


In [10]:
X = data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin'], 1)
y = data['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

model = LogisticRegression(random_state=123).fit(X_train, y_train)
print("LogisticRegression: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))


LogisticRegression: 
train score: 0.797904191617
test score:  0.798206278027


Получавам добро подобрение. Сега ще добавя и три колони за всяка от стойностите в Embarked.

In [11]:

data['S'] = (data.Embarked == 'S').astype(float)
data['C'] = (data.Embarked == 'C').astype(float)
data['Q'] = (data.Embarked == 'Q').astype(float)

data.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Male,Female,S,C,Q
PassengerId,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,1.0,0.0,1.0,0.0,0.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,0.0,1.0,0.0,1.0,0.0
3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,0.0,1.0,1.0,0.0,0.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,0.0,1.0,1.0,0.0,0.0
5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,1.0,0.0,1.0,0.0,0.0
6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q,1.0,0.0,0.0,0.0,1.0
7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,S,1.0,0.0,1.0,0.0,0.0
8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S,1.0,0.0,1.0,0.0,0.0
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,NaN,S,0.0,1.0,1.0,0.0,0.0


In [12]:
X = data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin'], 1)
y = data['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

model = LogisticRegression(random_state=123).fit(X_train, y_train)
print("LogisticRegression: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))


LogisticRegression: 
train score: 0.803892215569
test score:  0.780269058296


Получавам по-добър train score, но по-лош test score .За сега все пак ще ги оставя във фичърите за модела. Ще пробвам и с други алгоритми.

In [13]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier


X = data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin'], 1)
y = data['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

model = LogisticRegression(random_state=123).fit(X_train, y_train)
print("LogisticRegression: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = LinearSVC(random_state=123).fit(X_train, y_train)
print("LinearSVC: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = RandomForestClassifier(random_state=123).fit(X_train, y_train)
print("RandomForest: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = KNeighborsClassifier().fit(X_train, y_train)
print("KNeighborsClassifier: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))


LogisticRegression: 
train score: 0.803892215569
test score:  0.780269058296
LinearSVC: 
train score: 0.808383233533
test score:  0.780269058296
RandomForest: 
train score: 0.965568862275
test score:  0.816143497758
KNeighborsClassifier: 
train score: 0.812874251497
test score:  0.708520179372


Не получавам много по-добри резултати. Някои модели овърфитват.

Ще разделя Pclass в 3 колони тъй като номера на класата като число не е значим за резултата.

In [14]:
data['First'] = (data.Pclass == 1).astype(float)
data['Second'] = (data.Pclass == 2).astype(float)
data['Third'] = (data.Pclass == 3).astype(float)

И отново тествам.

In [15]:

X = data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin', 'Pclass'], 1)
y = data['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

model = LogisticRegression(random_state=123).fit(X_train, y_train)
print("LogisticRegression: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = LinearSVC(random_state=123).fit(X_train, y_train)
print("LinearSVC: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = RandomForestClassifier(random_state=123).fit(X_train, y_train)
print("RandomForest: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = KNeighborsClassifier().fit(X_train, y_train)
print("KNeighborsClassifier: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))


LogisticRegression: 
train score: 0.812874251497
test score:  0.784753363229
LinearSVC: 
train score: 0.699101796407
test score:  0.654708520179
RandomForest: 
train score: 0.971556886228
test score:  0.798206278027
KNeighborsClassifier: 
train score: 0.811377245509
test score:  0.713004484305


При LinearSVC скора доста падна. При другите има съвсем малко подобрение.
Ще пробвам да нормализирам данните.

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

model = LogisticRegression(random_state=123).fit(X_train, y_train)
print("LogisticRegression: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = LinearSVC(random_state=123).fit(X_train, y_train)
print("LinearSVC: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = RandomForestClassifier(random_state=123).fit(X_train, y_train)
print("RandomForest: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = KNeighborsClassifier().fit(X_train, y_train)
print("KNeighborsClassifier: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))


LogisticRegression: 
train score: 0.812874251497
test score:  0.784753363229
LinearSVC: 
train score: 0.805389221557
test score:  0.798206278027
RandomForest: 
train score: 0.971556886228
test score:  0.798206278027
KNeighborsClassifier: 
train score: 0.859281437126
test score:  0.789237668161


Получавам доста по-добри резултати при LinearSVC и KNN. При другите няма промяна.
Сега от колоната с имената на пътниците ще извлека техните титли.

In [17]:
data['Title'] = data.Name.str.extract('([A-Za-z]+)\.', expand=False)
data.loc[data.Title == 'Mlle', 'Title'] = 'Miss'
data.loc[data.Title == 'Mme', 'Title']  = 'Mrs'
data.loc[data.Title == 'Ms', 'Title']   = 'Miss'

rare_titles = ['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Don', 'Jonkheer', 'Capt', 'Lady','Sir']
data.Title = data.Title.replace(rare_titles, 'Rare')      

data.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Rare'], dtype=object)

Ще добавя титлите като фичъри и ще пробвам дали това ще окаже влияние на резултата.

In [18]:
data['Mr'] = (data.Title == 'Mr').astype(float)
data['Mrs'] = (data.Title == 'Mrs').astype(float)
data['Miss'] = (data.Title == 'Miss').astype(float)
data['Master'] = (data.Title == 'Master').astype(float)
data['Rare'] = (data.Title == 'Rare').astype(float)

In [19]:
X = data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin', 'Pclass', 'Title'], 1)
y = data['Survived']

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

model = LogisticRegression(random_state=123).fit(X_train, y_train)
print("LogisticRegression: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = LinearSVC(random_state=123).fit(X_train, y_train)
print("LinearSVC: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = RandomForestClassifier(random_state=123).fit(X_train, y_train)
print("RandomForest: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

model = KNeighborsClassifier().fit(X_train, y_train)
print("KNeighborsClassifier: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))


LogisticRegression: 
train score: 0.838323353293
test score:  0.793721973094
LinearSVC: 
train score: 0.835329341317
test score:  0.807174887892
RandomForest: 
train score: 0.971556886228
test score:  0.820627802691
KNeighborsClassifier: 
train score: 0.866766467066
test score:  0.807174887892


Има подобрение с около 0.01.
Забелязвам, че титли като Miss, Mrs и Master носят информация за приблизителната възраст на хората.
Ще използвам средната възраст за всяка титла за да попълня липсващата възраст на някои от пътниците.

In [20]:
data.Age = original.Age

age_by_title = data.groupby('Title').Age.mean()

data.loc[data.Age.isnull() & (data.Title == 'Mr'), 'Age'] = age_by_title['Mr']
data.loc[data.Age.isnull() & (data.Title == 'Mrs'), 'Age'] = age_by_title['Mrs']
data.loc[data.Age.isnull() & (data.Title == 'Miss'), 'Age'] = age_by_title['Miss']
data.loc[data.Age.isnull() & (data.Title == 'Master'), 'Age'] = age_by_title['Master']
data.loc[data.Age.isnull() & (data.Title == 'Rare'), 'Age'] = age_by_title['Rare']   

Ще добавя няколко колони относно това дали човек е сам или с малко/голямо семейство. Информацията за това се съдържа в колоните Parch и SibSp.

In [21]:
data['Alone'] = (data.SibSp + data.Parch == 0).astype(float)
data['SmallFamily'] = ((data.SibSp + data.Parch > 0) & (data.SibSp + data.Parch <= 3)).astype(float)
data['BigFamily'] = (data.SibSp + data.Parch >= 4).astype(float)



Крайно време е да отделя кода с тестването на различните алгоритми в отделна функция.

In [22]:
def test(X, y):
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

    model = LogisticRegression(random_state=123).fit(X_train, y_train)
    print("LogisticRegression: ")
    print("train score:", model.score(X_train, y_train))
    print("test score: ", model.score(X_test, y_test))

    model = LinearSVC(random_state=123).fit(X_train, y_train)
    print("LinearSVC: ")
    print("train score:", model.score(X_train, y_train))
    print("test score: ", model.score(X_test, y_test))

    model = RandomForestClassifier(random_state=123).fit(X_train, y_train)
    print("RandomForest: ")
    print("train score:", model.score(X_train, y_train))
    print("test score: ", model.score(X_test, y_test))

    model = KNeighborsClassifier().fit(X_train, y_train)
    print("KNeighborsClassifier: ")
    print("train score:", model.score(X_train, y_train))
    print("test score: ", model.score(X_test, y_test))


In [23]:
test(data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin', 'Pclass', 'Title'], 1), data.Survived)

LogisticRegression: 
train score: 0.833832335329
test score:  0.816143497758
LinearSVC: 
train score: 0.835329341317
test score:  0.816143497758
RandomForest: 
train score: 0.968562874251
test score:  0.811659192825
KNeighborsClassifier: 
train score: 0.86377245509
test score:  0.793721973094


Отново леко подобрение при някои от алгоритмите.
Според филма това да има дете с пътника доста помага за неговото качване на спасителна лодка.
Ще се опитам да пресметна броя на децата с човек като използвам номера на билета му.

In [24]:
l = []
for index, row in data.iterrows():
    c = data[(data.Ticket == row['Ticket']) & (data.Age <= 14)]
    a = data[(data.Ticket == row['Ticket']) & (data.Age > 14)]
    if(row.Age > 14):
        l.insert(index, float(len(c.index)/len(a.index)))
    else:
        l.insert(index, 0.0)
        
data['WithChildren'] = l

test(data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin', 'Pclass', 'Title'], 1), data.Survived)

LogisticRegression: 
train score: 0.833832335329
test score:  0.816143497758
LinearSVC: 
train score: 0.833832335329
test score:  0.816143497758
RandomForest: 
train score: 0.974550898204
test score:  0.80269058296
KNeighborsClassifier: 
train score: 0.860778443114
test score:  0.798206278027


Резултат на RandomForest стана по нисък за тест сета.
За сега няма да използвам този фичър.
Ще опитам да разделя пасажерите на няколко възрастови групи и да използвам тях вместо колоната Age.

In [25]:
data['Child'] = (data.Age <= 12).astype(float)
data['MidAge'] = ((data.Age >= 13) & (data.Age <= 60)).astype(float)
data['Old'] = (data.Age > 60).astype(float)

Освен това ще премахна децата от колоните male и female, защото полът на децата надали е бил от значение и може би създава грешни зависимости.
Ще разделя и колоната Fare на 2 колони LowFare и HighFare.

In [26]:
data['Male'] = ((data.Sex == 'male') & (data.Age>12)).astype(float)
data['Female'] = ((data.Sex == 'female') & (data.Age>12)).astype(float)
data['LowFare'] = (data.Fare<data.Fare.mean()).astype(float)
data['HignFare'] = (data.Fare>=data.Fare.mean()).astype(float)

test(data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin', 'Pclass', 'Title', 'Age', 'WithChildren','S','Q','C'], 1), data.Survived)

LogisticRegression: 
train score: 0.832335329341
test score:  0.816143497758
LinearSVC: 
train score: 0.833832335329
test score:  0.816143497758
RandomForest: 
train score: 0.929640718563
test score:  0.766816143498
KNeighborsClassifier: 
train score: 0.865269461078
test score:  0.816143497758


Ще пробвам да променя хиперпараметрите в алгоритмите. ще използвам GridSearch за да ги подбера.

In [27]:
from sklearn.model_selection import GridSearchCV


X=data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin', 'Pclass', 'Title', 'Age', 'WithChildren','S','Q','C'], 1)
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

gs = GridSearchCV(LogisticRegression(),{'C':[1, 2,3,6,8,10,20,30,40,50,100,200]})
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)[['rank_test_score', 'mean_test_score', 'params']].sort_values(by='rank_test_score').head(10)

,rank_test_score,mean_test_score,params
0,1,0.830838,{'C': 1}
4,2,0.829341,{'C': 8}
5,2,0.829341,{'C': 10}
6,2,0.829341,{'C': 20}
7,2,0.829341,{'C': 30}
8,2,0.829341,{'C': 40}
9,2,0.829341,{'C': 50}
10,2,0.829341,{'C': 100}
11,2,0.829341,{'C': 200}
1,10,0.827844,{'C': 2}


Най-добър резултат получих с логистична регресия с С=1.

In [28]:
X=data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin', 'Pclass', 'Title', 'Age', 'WithChildren','S','Q','C'], 1)
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

gs = GridSearchCV(RandomForestClassifier(),{'n_estimators':[1, 5,10,20,30,40,50,100,200,250,300]})
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)[['rank_test_score', 'mean_test_score', 'params']].sort_values(by='rank_test_score').head(10)

,rank_test_score,mean_test_score,params
5,1,0.820359,{'n_estimators': 40}
4,2,0.818862,{'n_estimators': 30}
6,3,0.817365,{'n_estimators': 50}
7,3,0.817365,{'n_estimators': 100}
8,5,0.812874,{'n_estimators': 200}
9,6,0.809880,{'n_estimators': 250}
10,6,0.809880,{'n_estimators': 300}
1,8,0.796407,{'n_estimators': 5}
2,9,0.794910,{'n_estimators': 10}
3,10,0.791916,{'n_estimators': 20}


За RandomForest получих стойност за n_estimators 40.

In [29]:
X=data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin', 'Pclass', 'Title', 'Age', 'WithChildren','S','Q','C'], 1)
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

gs = GridSearchCV(KNeighborsClassifier(),{'n_neighbors':[1, 5,10,20,30,40,50,100,200]})
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)[['rank_test_score', 'mean_test_score', 'params']].sort_values(by='rank_test_score').head(10)

,rank_test_score,mean_test_score,params
1,1,0.772455,{'n_neighbors': 5}
2,2,0.738024,{'n_neighbors': 10}
0,3,0.735030,{'n_neighbors': 1}
3,4,0.727545,{'n_neighbors': 20}
4,5,0.706587,{'n_neighbors': 30}
6,6,0.685629,{'n_neighbors': 50}
5,7,0.682635,{'n_neighbors': 40}
7,8,0.673653,{'n_neighbors': 100}
8,9,0.633234,{'n_neighbors': 200}


In [30]:
X=data.drop(['Survived', 'Sex', 'Embarked', 'Name', 'Ticket', 'Cabin', 'Pclass', 'Title', 'Age', 'WithChildren','S','Q','C'], 1)
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

gs = GridSearchCV(LinearSVC(),{'C':[1, 2,3,6,8,10,20,30,40,50,100,200]})
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)[['rank_test_score', 'mean_test_score', 'params']].sort_values(by='rank_test_score').head(10)

,rank_test_score,mean_test_score,params
0,1,0.812874,{'C': 1}
3,2,0.811377,{'C': 6}
1,3,0.791916,{'C': 2}
2,4,0.785928,{'C': 3}
11,5,0.748503,{'C': 200}
7,6,0.742515,{'C': 30}
8,7,0.733533,{'C': 40}
10,8,0.694611,{'C': 100}
4,9,0.690120,{'C': 8}
6,10,0.675150,{'C': 20}


Не получих особена промяна в хиперпараметрите. Ще отделя всички трансформации в отделна функция.

In [31]:
def transform(data):
    data2 = data.copy()
    data2['FamilySize'] = data2['Parch'] + data2['SibSp'] + 1
    data2['Alone'] = (data2.SibSp + data2.Parch == 0).astype(float)
    data2['SmallFamily'] = ((data2.SibSp + data2.Parch > 0) & (data2.SibSp + data2.Parch <= 3)).astype(float)
    data2['BigFamily'] = (data2.SibSp + data2.Parch >= 4).astype(float)

    data2['Title'] = data2.Name.str.extract('([A-Za-z]+)\.', expand=False)
    data2.loc[data2.Title == 'Mlle', 'Title'] = 'Miss'
    data2.loc[data2.Title == 'Mme', 'Title']  = 'Mrs'
    data2.loc[data2.Title == 'Ms', 'Title']   = 'Miss'
    
    rare_titles = ['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Don', 'Jonkheer', 'Capt', 'Lady','Sir']
    data2.Title = data2.Title.replace(rare_titles, 'Rare')      
    age_by_title = data2.groupby('Title').Age.mean()
    
    data2['Miss'] = (data2.Title == 'Miss').astype(float)
    data2['Mrs'] = (data2.Title == 'Mrs').astype(float)
    data2['Mr'] = (data2.Title == 'Mr').astype(float)
    data2['Lady'] = (data2.Title == 'Lady').astype(float)
    data2['Sir'] = (data2.Title == 'Sir').astype(float)
    data2['Rare'] = (data2.Title == 'Rare').astype(float)
    
    data2.loc[data2.Age.isnull() & (data2.Title == 'Mr'), 'Age'] = age_by_title['Mr']
    data2.loc[data2.Age.isnull() & (data2.Title == 'Mrs'), 'Age'] = age_by_title['Mrs']
    data2.loc[data2.Age.isnull() & (data2.Title == 'Miss'), 'Age'] = age_by_title['Miss']
    data2.loc[data2.Age.isnull() & (data2.Title == 'Master'), 'Age'] = age_by_title['Master']
    data2.loc[data2.Age.isnull() & (data2.Title == 'Rare'), 'Age'] = age_by_title['Rare']   
    
    data2['Child'] = (data2.Age <= 12).astype(float)
    data2['MidAge'] = ((data2.Age >= 13) & (data2.Age <= 60)).astype(float)
    data2['Old'] = (data2.Age > 60).astype(float)                                  
    data2['Male'] = ((data2.Sex == 'male') & (data2.Age>12)).astype(float)
    data2['Female'] = ((data2.Sex == 'female') & (data2.Age>12)).astype(float)
    data2['First'] = ((data2.Pclass == 1)).astype(float)
    data2['Second'] = ((data2.Pclass == 2)).astype(float)
    data2['Third'] = ((data2.Pclass == 3)).astype(float)
    
    data2['LowFare'] = (data2.Fare<data2.Fare.mean()).astype(float)
    data2['HignFare'] = (data2.Fare>=data2.Fare.mean()).astype(float)
    
    data2 = data2.drop(['Sex', 'Title', 'Embarked', 'Cabin', 'Name','Ticket', 'SibSp','Fare', 'Age','Parch','FamilySize', 'Pclass'],1)
    return data2

Ще пробвам дали всичко работи правилно и ще опитам с cross validation да видя какво се е получило.

In [32]:
from sklearn.neighbors import KNeighborsClassifier

data=original.copy()
X=transform(data.drop('Survived',1))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

model = LogisticRegression(random_state=123).fit(X_train, y_train)
print("LogisticRegression: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

print(cross_val_score(model, X,y))

model = LinearSVC(random_state=123).fit(X_train, y_train)
print("LinearSVC: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

print(cross_val_score(model, X,y))


model = RandomForestClassifier(random_state=123, n_estimators=40).fit(X_train, y_train)
print("RandomForest: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

print(cross_val_score(model, X,y))

model = KNeighborsClassifier(n_neighbors=50).fit(X_train, y_train)
print("KNeighborsClassifier: ")
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))


print(cross_val_score(model, X,y))


LogisticRegression: 
train score: 0.836826347305
test score:  0.816143497758
[ 0.82154882  0.80808081  0.82828283]
LinearSVC: 
train score: 0.830838323353
test score:  0.816143497758
[ 0.82154882  0.83501684  0.82491582]
RandomForest: 
train score: 0.850299401198
test score:  0.807174887892
[ 0.81818182  0.84175084  0.84175084]
KNeighborsClassifier: 
train score: 0.824850299401
test score:  0.811659192825
[ 0.80808081  0.83838384  0.80808081]


След малко тестване получих най-добри резултати с KNN с 50 съседа.
Сега ще тренирам модела с всички тренировъчни данни и ще пусна предсказанията за тестовите данни в kaggle.

In [33]:
train = pd.read_csv('data/titanic/train.csv',index_col=['PassengerId'])
test = pd.read_csv('data/titanic/test.csv',index_col=['PassengerId'])

X = transform(data.drop('Survived',1));
test = transform(test)

model = KNeighborsClassifier(n_neighbors=50).fit(X, train['Survived'])

predictions = model.predict(test)
frame = pd.DataFrame({
    'PassengerId': pd.read_csv('data/titanic/test.csv').PassengerId,
    'Survived': predictions
})
frame = frame.set_index('PassengerId')
frame.to_csv('data/titanic/predictions.csv')
frame.head(10)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
897,0
898,1
899,0
900,1


Получих резултат в kaggle 0.79904 (1693/9359). Не е много по-добър, но все пак има подобрение.
Същия резултат получих и с RandomForestClassifier(n_estimators=40).


In [34]:
X.head()

,Alone,SmallFamily,BigFamily,Miss,Mrs,Mr,Lady,Sir,Rare,Child,MidAge,Old,Male,Female,First,Second,Third,LowFare,HignFare
PassengerId,,,,,,,,,,,,,,,,,,,
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


Във финалния модел използвах фичърите:
    - за семейство на пасажера: Alone, SmallFamily, BigFamily
    - за титла: Miss, Mrs, Mr, Rare
    - за възраст: Child, MidAge, Old
    - за пол: Male, Female(без децата)
    - за класа: First, Second, Third
    - за цена на билета: LowFare, HignFare
        
и KNeighborsClassifier(n_neighbors=50).